Load learner

In [ ]:
from fastai.vision.all import load_learner

learner = load_learner("export.pkl")

In [ ]:
# Pretty evaluation with image display
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path

# Test images
test_images = [
    ("data/valid/linear/18.png", "linear"),
    ("data/valid/cubic/18.png", "cubic"),
    ("data/valid/quadratic/18.png", "quadratic"),
]

# Create subplot for all 3 images
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("Graph Classification Results", fontsize=16, fontweight="bold")

for i, (img_path, true_label) in enumerate(test_images):
    # Make prediction
    pred_class, pred_idx, probs = learner.predict(img_path)

    # Load and display image
    img = Image.open(img_path)
    axes[i].imshow(img)
    axes[i].axis("off")

    # Check if correct
    is_correct = str(pred_class) == true_label
    confidence = probs.max().item()

    # Color title based on correctness
    color = "green" if is_correct else "red"

    # Set image title
    title = f"True: {true_label}\nPred: {pred_class}\nConf: {confidence:.1%}"
    axes[i].set_title(title, fontsize=12, color=color, fontweight="bold")

    # Print detailed results
    print(f"\n{'=' * 40}")
    print(f"📸 IMAGE: {Path(img_path).name}")
    print(f"{'=' * 40}")
    print(f"True Label:    {true_label}")
    print(f"Predicted:     {pred_class}")
    print(f"Confidence:    {confidence:.1%}")
    print(f"Status:        {'✅ CORRECT' if is_correct else '❌ WRONG'}")
    print("\nProbability Breakdown:")
    print(f"{'Class':<12} {'Probability':<12} {'Bar'}")
    print(f"{'-' * 40}")

    for cls, prob in zip(learner.dls.vocab, probs):
        prob_val = prob.item()
        bar_length = int(prob_val * 30)
        bar = "█" * bar_length + "░" * (30 - bar_length)
        print(f"{cls:<12} {prob_val:>7.1%}       {bar}")

plt.tight_layout()
plt.show()